# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [6]:
# load libraries
import pandas as pd
import numpy as np

In [7]:
# Q1: Find the top 10 countries with the lowest infant mortality rates
results = []

for element in document.iterfind('country'):
    name = element.find('name').text
    infant_mortality = element.find("infant_mortality")
    
    if infant_mortality is not None:
        infant_mortality = infant_mortality.text
    else:
        infant_mortality = None

    results.append([name, infant_mortality])

results_pd = pd.DataFrame(results) # convert list to pandas DataFrame
results_pd.rename(columns={0: 'Country', 1: 'Infant_mortality'}, inplace=True) # rename columns 

results_pd['Country'] = results_pd['Country'].astype(str) # convert Country to appropriate format
results_pd['Infant_mortality'] = results_pd['Infant_mortality'].astype(float) # Convert Infant_mortality to appropriate format

#results_pd.dtypes # check types
results_pd.sort_values(by = ['Infant_mortality'], ascending=['TRUE']).head(10) #get top 10 countries with lowest IM rate

,Country,Infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [8]:
# Q2: Find the top 10 countries with the largest population
results = []

for element in document.iterfind('country'):
    name = element.find('name').text
    
    i = 2016
    while element.find(''.join(["population[@year='", str(i) ,"']"])) is None:
        i = i - 1
    population = element.find(''.join(["population[@year='", str(i) ,"']"])).text

    results.append([name, population])

results_pd = pd.DataFrame(results) # convert list to pandas DataFrame
results_pd.rename(columns={0: 'Country', 1: 'Population'}, inplace = True) # rename columns 

results_pd['Country'] = results_pd['Country'].astype(str) # convert Country to appropriate format
results_pd['Population'] = results_pd['Population'].astype(float) # Convert Population to appropriate format

#results_pd.dtypes # check types
results_pd.sort_values(by = ['Population'], ascending=False).head(10) #get top 10 countries with the highest population

,Country,Population
55,China,1.360720e+09
67,India,1.210855e+09
120,United States,3.188571e+08
88,Indonesia,2.521245e+08
176,Brazil,2.027686e+08
57,Pakistan,1.731493e+08
202,Nigeria,1.642945e+08
65,Bangladesh,1.497724e+08
23,Russia,1.436669e+08
98,Japan,1.272980e+08


In [9]:
# Q3: Find the top 10 ethnic groups with the largest overall populations 
# (sum of best/latest estimates over all countries)
results = []

for element in document.iterfind('country'):
    Country = element.find('name').text
    
    i = 2016
    while element.find(''.join(["population[@year='", str(i) ,"']"])) is None:
        i = i - 1
    Population = element.find(''.join(["population[@year='", str(i) ,"']"])).text

    # The same ethnic group can appear in multiple countries
    j = 1
    while element.find(''.join(["./ethnicgroup[", str(j) ,"]"])) is not None:
        if element.find(''.join(["./ethnicgroup[", str(j) ,"]"])) is not None:
            EthnicGroup = element.find(''.join(["./ethnicgroup[", str(j) ,"]"])).text
            Percentage = float(element.find(''.join(["./ethnicgroup[", str(j) ,"]"])).get('percentage'))/100
            results.append([Country, Population, EthnicGroup, Percentage])
        else:
            EthnicGroup = None
            Percentage = None
            results.append([Country, Population, EthnicGroup, Percentage])
        j = j + 1

results_pd = pd.DataFrame(results) # convert list to pandas DataFrame
results_pd.rename(columns={0: 'Country', 1: 'Population', 2: 'Ethnic_group', 3: 'Percentage'}, inplace = True) # rename columns 

results_pd['Country'] = results_pd['Country'].astype(str) # convert Country to appropriate format
results_pd['Population'] = results_pd['Population'].astype(float) # Convert Population to appropriate format
results_pd['Ethnic_group'] = results_pd['Ethnic_group'].astype(str) # convert Ethnic Group to appropriate format
results_pd['Percentage'] = results_pd['Percentage'].astype(float) # Convert Percentage to appropriate format

results_pd['Ethnic_population'] = results_pd['Population']*results_pd['Percentage'] # Calculate ethnic population in each country

results_pd = results_pd.groupby(by=results_pd['Ethnic_group']).sum().drop(['Percentage'], axis = 1) # calculate ethnic population size
results_pd.sort_values(by = ['Ethnic_population'], ascending=False).head(11) #get top 10 ethnic groups with the highest population

,Population,Ethnic_population
Ethnic_group,,
Han Chinese,1.360720e+09,1.245059e+09
Indo-Aryan,1.210855e+09,8.718156e+08
European,1.157296e+09,4.948722e+08
African,9.753527e+08,3.183251e+08
Dravidian,1.210855e+09,3.027137e+08
Mestizo,2.797440e+08,1.577344e+08
Bengali,1.497724e+08,1.467769e+08
Russian,3.224384e+08,1.318570e+08
Japanese,1.272980e+08,1.265342e+08


In [10]:
#Q4a. Find the country with and the name of the longest river
results = []

for element in document.iterfind('river'):
    River  = element.find('name').text
    Country = element.get('country')
    
    if element.find('length') is not None:
        Length = element.find('length').text
    else:
        Length = None
    
    results.append([River, Length, Country])

results_pd = pd.DataFrame(results) # convert list to pandas DataFrame
results_pd.rename(columns={0: 'River', 1: 'Length', 2: 'Country'}, inplace = True) # rename columns 

results_pd['River'] = results_pd['River'].astype(str) # convert River to appropriate format
results_pd['Length'] = results_pd['Length'].astype(float) # Convert Length to appropriate format
results_pd['Country'] = results_pd['Country'].astype(str) # convert Country to appropriate format

results_pd.sort_values(by = ['Length'], ascending=False).head(10) #get top 10 longest rivers

,River,Length,Country
174,Amazonas,6448.0,CO BR PE
137,Jangtse,6380.0,CN
136,Hwangho,4845.0,CN
123,Lena,4400.0,R
205,Zaire,4374.0,RCB ZRE
138,Mekong,4350.0,CN LAO THA K VN
115,Irtysch,4248.0,R KAZ CN
186,Niger,4184.0,RMM RN WAN RG
160,Missouri,4130.0,USA
119,Jenissej,4092.0,R


In [11]:
#Q4b. Find the country with and the name of the largest lake
results = []

for element in document.iterfind('lake'):
    Lake  = element.find('name').text
    Country = element.get('country')
    
    if element.find('area') is not None:
        Area = element.find('area').text
    else:
        Area = None
    
    results.append([Lake, Area, Country])

results_pd = pd.DataFrame(results) # convert list to pandas DataFrame
results_pd.rename(columns={0: 'Lake', 1: 'Area', 2: 'Country'}, inplace = True) # rename columns 

results_pd['Lake'] = results_pd['Lake'].astype(str) # convert Lake to appropriate format
results_pd['Area'] = results_pd['Area'].astype(float) # convert Area to appropriate format
results_pd['Country'] = results_pd['Country'].astype(str) # convert Country to appropriate format

results_pd.sort_values(by = ['Area'], ascending=False).head(10) # get top 10 largest lakes

,Lake,Area,Country
54,Caspian Sea,386400.0,R AZ KAZ IR TM
109,Lake Superior,82103.0,CDN USA
81,Lake Victoria,68870.0,EAT EAK EAU
106,Lake Huron,59600.0,CDN USA
108,Lake Michigan,57800.0,USA
47,Dead Sea,41650.0,IL JOR WEST
83,Lake Tanganjika,32893.0,ZRE Z BI EAT
98,Great Bear Lake,31792.0,CDN
43,Ozero Baikal,31492.0,R
89,Lake Malawi,29600.0,MW MOC EAT


In [12]:
#Q4c. Find the country with and the name of the airport at highest elevation
results = []

for element in document.iterfind('airport'):
    Airport  = element.find('name').text
    Country = element.get('country')
    
    if element.find('elevation') is not None:
        Elevation = element.find('elevation').text
    else:
        Elevation = None
    
    results.append([Airport, Elevation, Country])

results_pd = pd.DataFrame(results) # convert list to pandas DataFrame
results_pd.rename(columns={0: 'Airport', 1: 'Elevation', 2: 'Country'}, inplace = True) # rename columns 

results_pd['Airport'] = results_pd['Airport'].astype(str) # convert airport to appropriate format
results_pd['Elevation'] = results_pd['Elevation'].astype(float) # convert elevation to appropriate format
results_pd['Country'] = results_pd['Country'].astype(str) # convert Country to appropriate format

results_pd.sort_values(by = ['Elevation'], ascending=False).head(10) # get top 10 airport with the highest elevation

,Airport,Elevation,Country
80,El Alto Intl,4063.0,BOL
219,Lhasa-Gonggar,4005.0,CN
241,Yushu Batang,3963.0,CN
813,Juliaca,3827.0,PE
815,Teniente Alejandro Velasco Astete Intl,3311.0,PE
82,Juana Azurduy De Padilla,2905.0,BOL
334,Mariscal Sucre Intl,2813.0,EC
805,Coronel Fap Alfredo Mendivil Duarte,2719.0,PE
807,Mayor General FAP Armando Revoredo Iglesias Ai...,2677.0,PE
692,Licenciado Adolfo Lopez Mateos Intl,2581.0,MEX
